# hightlight_dataframe

In [1]:
import pandas as pd
from aiprint.core import hightlight_dataframe
raw = {}
for idx in range(5):
    col = f"col_{idx}"
    raw[col] = [1, 2, 3, 4, 5]
data = pd.DataFrame(raw)
hightlight_dataframe(data, "col_1 % 2 == 0")

   col_0  col_1  col_2  col_3  col_4
0      1      1      1      1      1
1      2      2      2      2      2
2      3      3      3      3      3
3      4      4      4      4      4
4      5      5      5      5      5


# highlight_dict

In [1]:
from aiprint.core import highlight_dict

data = {"success": False, "other_field": 123}
highlight_dict(data, "success")

pattern:  "success": (\w+)
{"success": False, "other_field": 123}


In [3]:
from aiprint.core import highlight_dict

data = {"success": 1, "other_field": 123}
highlight_dict(data, "success")

pattern:  "success": (\w+)
{"success": 1, "other_field": 123}


# highlight_text

In [1]:
from aiprint.core import highlight_text
logs = "Model status: failed. Cause: corrupted input data. All-other-related-stuffs"
highlight_text(logs, "status")

Model status: failed. Cause: corrupted input data. All-other-related-stuffs


In [2]:
logs = "Model status: true. Cause: corrupted input data. All-other-related-stuffs"
highlight_text(logs, "status")

Model status: true. Cause: corrupted input data. All-other-related-stuffs


# cprint

In [33]:
"""Core functions"""
from typing import Any

import pandas as pd
from logzero import logger
from termcolor import colored

from aiprint.helper import add_color, extract_status


def hightlight_dataframe(data: pd.DataFrame, condition: str, color: str = "red"):
    """Highlight rows in a dataframe based-on a given condition

    Examples
    --------
    import pandas as pd
    data = pd.DataFrame({"val": [7, 8, 9, 10], "col1": [10, 20, 30, 40], "col2": [1, 2, 3, 4]})

    # check one condition
    hightlight_dataframe(data, "val % 2 == 0")

    # check multiple conditions
    hightlight_dataframe(data, "col1 >= 20 & col2 <= 3")


    # check very large dataframe with fragmented print
    raw = {}
    for idx in range(30):
        col = f"col_{idx}"
        raw[col] = [1, 2, 3, 4, 5]
    data = pd.DataFrame(raw)
    hightlight_dataframe(data, "col_1 % 2 == 0")
    """
    mask = data.eval(condition)
    matched_idx = data.index[mask].tolist()
    partitioned_lines = str(data).split("\n\n")

    # to handle when very large dataframe is printed into more than 2 sections
    # , one parasite [n row x m col] is added
    if len(partitioned_lines) > 2:
        partitioned_lines = partitioned_lines[:-1]
    colored_lines = []
    for partitioned_str in partitioned_lines:
        lines = partitioned_str.split("\n")
        for idx in matched_idx:
            idx_line = idx + 1
            lines[idx_line] = colored(lines[idx_line], color)
        colored_lines.append("\n".join(lines))
    print("\n\n".join(colored_lines))


def highlight_dict(data: dict, key: str):
    """Highlight a given field of the dictionary

    Examples
    --------
    data = {"success": False,
            "other_field": 123}
    highlight_dict(data, "success")
    """
    data_str = str(data).replace("'", '"')
    to_extract = "(\s?\w+)"
    pattern = f'"{key}":{to_extract}'
    try:
        status = extract_status(data_str, pattern)
        filled_pattern = pattern.replace(to_extract, status)
        print(add_color(data_str, filled_pattern, status))
    except TypeError as emsg:
        logger.debug("in dict-item highlight mode.")
        highlight_dict_item(data, key)


def highlight_dict_item(data: dict, key: str, color: str = "yellow"):
    """Highlight a given item of the dictionary"""
    data_str = str(data)
    item = data[key]
    to_colors = [key, str(item)]
    colored_text = [colored(txt, color) for txt in to_colors]
    mapper = zip(to_colors, colored_text)
    for to_color, colored_txt in mapper:
        data_str = data_str.replace(to_color, colored_txt)
    print(data_str)


def highlight_text(data: dict, key: str):
    """Highlight a given field of the dictionary

    Examples
    --------
    logs = "Model status: failed. Cause: corrupted input data.
            All-other-related-stuffs"
    highlight_text(logs, "status")
    """
    data_str = str(data).replace("'", '"')
    """
    to_extract = "(\s?\w+)"
    sep = "\s?[:=]"
    pattern = f"{key}{sep}{to_extract}"
    print("pattern: ", pattern)
    """
    to_extract = "\w+"
    sep = "[:=]?"
    query = f"(\s?{sep}\s?{to_extract})"
    pattern = f"{key}{query}"
    extracted = extract_status(data_str, pattern)
    status = split_status(extracted, [":","="])
    filled_pattern = pattern.replace(query, extracted)
    print(add_color(data_str, filled_pattern, status))

def split_status(text: str, separators: None):
    """Split status from `sep`"""
    default_seps = [":","="]
    seps = separators if separators else default_seps
    for sep in seps:
        text = text.split(sep)[-1]
    return text

def cprint(data: Any, key: str):
    """print with colorcode for queried key"""
    mapper = {pd.DataFrame: hightlight_dataframe, dict: highlight_dict, str: highlight_text}
    mapper[type(data)](data, key)


In [34]:
#from aiprint.core import cprint
logs = "Model status: failed. Cause: corrupted input data. All-other-related-stuffs"
cprint(logs, "status")

Model status: failed. Cause: corrupted input data. All-other-related-stuffs


In [38]:
logs = "Model status = true. Cause: corrupted input data. All-other-related-stuffs"
cprint(logs, "status")

Model status = true. Cause: corrupted input data. All-other-related-stuffs


In [30]:
#txt = ": failed"
#txt = "= failed"
txt = " failed"
for sep in [":","="]:
    txt = txt.split(sep)[-1]

In [4]:
from aiprint.core import cprint
logs = "Model status= failed. Cause: corrupted input data. All-other-related-stuffs"
cprint(logs, "status")

Model status= failed. Cause: corrupted input data. All-other-related-stuffs


In [5]:
data = {"success": 1, "other_field": 123}
cprint(data, "success")

{"success": 1, "other_field": 123}


In [6]:
data = {"success":0, "other_field": 123}
cprint(data, "success")

{"success": 0, "other_field": 123}


In [5]:
import pandas as pd
raw = {}
for idx in range(5):
    col = f"col_{idx}"
    raw[col] = [1, 2, 3, 4, 5]
data = pd.DataFrame(raw)
cprint(data, "col_1 % 2 == 0")

   col_0  col_1  col_2  col_3  col_4
0      1      1      1      1      1
1      2      2      2      2      2
2      3      3      3      3      3
3      4      4      4      4      4
4      5      5      5      5      5


In [6]:
from aiprint.core import highlight_dict
from termcolor import colored

a_dict = {
    "a": "hello",
    "b": "kitty"
}
highlight_dict(a_dict, "a")


[D 230320 18:36:46 helper:25] Can not extract status from key
[D 230320 18:36:46 core:69] in dict-item highlight mode.


{'a': 'hello', 'b': 'kitty'}
